maximo por musica tipo 5
se a quantidade  
se a musica so existe em uma playlist sozinha, não recomendar

teremos um limite maximo de musicas que vamos recomendar ao usuario, se o nosso conjunto passar disso, ja podemos retornar


- Para cada musica em um array de musicas, vamos olhar o consquente dele por ordem e incluir em um conjunto de musicas V
- vamos atribuir um numero igual de musicas recomendadas por musicas passadas V
- checar sempre se a musica que foi passada o seu consequente tem musicas parecidas, para tirar set e incluir outra V


In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import *
import os
import pickle
import math

dir = "datasets/train/2023_spotify_ds1.csv"

model_path = ''
targets_path = ''

def Load_Database(dir):
  DF = pd.read_csv(dir)
  return DF

def Model_train(dir):
    DF = Load_Database(dir)
    df_onehot = DF.groupby(['pid', 'track_name'])['track_name'].count().unstack().fillna(0)
    
    def encode_units(x):
        if x <= 0:
            return False
        if x >= 1:
            return True
    
    df_onehot = df_onehot.applymap(encode_units)
    frequent_itemsets = apriori(df_onehot, min_support = 0.04, use_colnames = True, verbose = 1)
    
    rules = association_rules(frequent_itemsets, metric = "lift", min_threshold=1)
    
    # se não houver o diretorio ou o arquivo pickle, criar um
    # if not os.path.exists("{model_path}"):
    #     os.makedirs("{model_path}")
    
    # Gerando o conjunto de regras
    rules.to_pickle(f"rules.pkl")


Model_train(dir)

Processing 1116192 combinations | Sampling itemset size 2


KeyboardInterrupt



In [4]:
with open("rules.pkl", 'rb') as rules_pickle:
    rules = pickle.load(rules_pickle)

In [5]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,($ave Dat Money (feat. Fetty Wap & Rich Homie ...,(Broccoli (feat. Lil Yachty)),0.067639,0.203360,0.041114,0.607843,2.989003,0.027359,2.031432,0.713715
1,(Broccoli (feat. Lil Yachty)),($ave Dat Money (feat. Fetty Wap & Rich Homie ...,0.203360,0.067639,0.041114,0.202174,2.989003,0.027359,1.168627,0.835308
2,(0 To 100 / The Catch Up),(Back To Back),0.092396,0.107427,0.041556,0.449761,4.186662,0.031630,1.622154,0.838633
3,(Back To Back),(0 To 100 / The Catch Up),0.107427,0.092396,0.041556,0.386831,4.186662,0.031630,1.480186,0.852755
4,(0 To 100 / The Catch Up),(Broccoli (feat. Lil Yachty)),0.092396,0.203360,0.041556,0.449761,2.211650,0.022766,1.447807,0.603621
...,...,...,...,...,...,...,...,...,...,...
55895,(Mask Off),"(HUMBLE., XO TOUR Llif3, Bad and Boujee (feat....",0.152078,0.044651,0.040230,0.264535,5.924534,0.033439,1.298973,0.980291
55896,(XO TOUR Llif3),"(HUMBLE., Bad and Boujee (feat. Lil Uzi Vert),...",0.151194,0.045093,0.040230,0.266082,5.900757,0.033412,1.301109,0.978468
55897,(T-Shirt),"(HUMBLE., XO TOUR Llif3, Bad and Boujee (feat....",0.223696,0.049956,0.040230,0.179842,3.600021,0.029055,1.158367,0.930336
55898,(Congratulations),"(HUMBLE., XO TOUR Llif3, Bad and Boujee (feat....",0.164898,0.049956,0.040230,0.243968,4.883675,0.031992,1.256619,0.952263


In [1]:
# song = ['Bohemian Rhapsody - Remastered 2011','Mask Off']
# for musica in song:
#   filter_rules = rules[rules['antecedents'].apply(lambda x: musica in x)]

In [2]:
# filter_rules

In [6]:
def playlist_recommender(musics, qtd):
  
  # Número de musicas para cada musica 
  n_per_music = math.ceil(qtd/len(musics))
  print(len(musics))

  # Carregando arquivo pickle
  with open("rules.pkl", 'rb') as rules_pickle:
    rules = pickle.load(rules_pickle)
    
    # Conjunto que será retornado no final
    playlist = set()

    # Para cada musica passada pelo usuário
    for song in musics:
      
      consequents = set()
      
      # Todos os elementos associados a uma musica
      filter_rules = rules[rules['antecedents'].apply(lambda x: song in x)]

      # Todos os consequentes de uma determinada musica
      for consequent in filter_rules['consequents']:
        # A cada iteração coloca todos os consequentes de uma musica em um consjunto
        consequents.update(consequent)
      
      # Transformado em uma lista para facilitar a manipulação
      consequents = list(consequents)
      
      contador = 0
      
      # Para cada musica na lista de consequentes 
      for music in consequents:
        # Contador para inserir uma quantidade x na playlist, a ausencia disso poderia fazer valores
        # duplica entrarem na playlist e não atingir a quantidade total de musicas.
        if contador == n_per_music:
          break
        # Se a musica não estiver na playlist, adicionar
        elif music not in playlist:
          playlist.add(music)
          contador = contador + 1
    
  return playlist

In [10]:
song = ['Mask Off']
play = playlist_recommender(song, 10)
play

1


{'Bad and Boujee (feat. Lil Uzi Vert)',
 'Black Beatles',
 "I Don't Fuck With You",
 'Magnolia',
 'Moves',
 'Panda',
 'Portland',
 'Too Many Years',
 'Wyclef Jean',
 'pick up the phone'}